# Choosing Appropriate Exposure Times and Dark-Frame Rules

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from bluesky_tutorial_utils import setup_data_saving
from bluesky import RunEngine
from bluesky.plans import count

RE = RunEngine()
catalog = setup_data_saving(RE)

We will import some simulated devices from a Python script in the current directory, [simulated_hardware.py](./simulated_hardware.py). The objects present a programmatic interface that looks like real hardware, but their readings from come functions defined in another script, [generate_data.py](./generate_data.py), instead of from actual measurements.

In [ ]:
from simulated_hardware import detector, shutter

In [ ]:
RE(count([detector]))

In [ ]:
data = catalog[-1].primary.read()
data

In [ ]:
plt.figure()

In [ ]:
plt.imshow(data["detector_image"].squeeze())
plt.colorbar()

In [ ]:
shutter.set("closed").wait()

In [ ]:
RE(count([detector]))

In [ ]:
dark_data = catalog[-1].primary.read()
dark_data

In [ ]:
plt.figure()

In [ ]:
plt.imshow(dark_data["detector_image"].squeeze())
plt.colorbar()

In [ ]:
from scipy import ndimage
import numpy as np

def simple_integration(image, num_bins=3001):
    sx, sy = image.shape
    X, Y = np.ogrid[0:sx, 0:sy]
    r = np.hypot(X - sx/2, Y - sy/2)
    rbin = (num_bins * r/r.max()).astype(np.int)
    radial_mean = ndimage.mean(image, labels=rbin, index=np.arange(1, rbin.max() + 1))
    return radial_mean

In [ ]:
intensity = simple_integration(data["detector_image"].squeeze() - dark_data["detector_image"].squeeze())

In [ ]:
plt.figure()

In [ ]:
plt.plot(intensity)

In [ ]:
plt.figure()

In [ ]:
(data["detector_image"].values - (dark_data["detector_image"] - 10).values).max()

In [ ]:
dark_data["detector_image"].min()